<a href="https://colab.research.google.com/github/palbha/llm_rag/blob/main/1_simple_rag_freeofcost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Simple RAG: Enhancing Language Models with External Knowledge

Retrieval-Augmented Generation (RAG) is a powerful technique that combines the strengths of information retrieval and generative models. By incorporating external knowledge sources, RAG significantly improves the accuracy and factual grounding of language models.

**Here's how Simple RAG works:**

1. **Data Ingestion:**  The process begins by loading and preprocessing the text data that will serve as the external knowledge base.
2. **Chunking:**  The data is then divided into smaller, manageable chunks to optimize retrieval efficiency.
3. **Embedding Creation:** Each text chunk is transformed into a numerical representation called an embedding using a pre-trained embedding model. These embeddings capture the semantic meaning of the text.
4. **Semantic Search:** When a user poses a query, a semantic search is performed using the query's embedding to identify the most relevant text chunks from the knowledge base.
5. **Response Generation:** Finally, a language model utilizes the retrieved chunks as context to generate a comprehensive and informative response to the user's query.

**In this notebook, we'll explore a basic implementation of Simple RAG, assess the quality of the generated responses, and discuss potential enhancements to further improve its performance.**

The motivation & contents in the notebook are derived from "https://github.com/FareedKhan-dev/all-rag-techniques/blob/main/1_simple_rag.ipynb" - All thanks to FareedKhan

In [ ]:
#Install relevany libraries - The code can run on Google colab
!pip install faiss-cpu

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
def get_text(file_path):
  with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        return text
def chunk_text(text, n, overlap):
    """
    Chunks the given text into segments of n characters with overlap.

    Args:
    text (str): The text to be chunked.
    n (int): The number of characters in each chunk.
    overlap (int): The number of overlapping characters between chunks.

    Returns:
    List[str]: A list of text chunks.
    """
    chunks = []  # Initialize an empty list to store the chunks

    # Loop through the text with a step size of (n - overlap)
    for i in range(0, len(text), n - overlap):
        # Append a chunk of text from index i to i + n to the chunks list
        chunks.append(text[i:i + n])

    return chunks  # Return the list of text chunks



def create_embeddings(text, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    """
    Creates embeddings for the given text using a Hugging Face model.

    Args:
    text (str): The input text for which embeddings are to be created.
    model_name (str): The model to be used for creating embeddings. Default is "sentence-transformers/all-MiniLM-L6-v2".

    Returns:
    dict: A dictionary containing the embeddings.
    """
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Generate the embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the embeddings (mean pooling of the last hidden state)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

    return {"embeddings": embeddings}



In [ ]:
pdf_path = "/content/25_new.txt"

# Extract text from the PDF file
extracted_text = get_text(pdf_path)

# Chunk the extracted text into segments of 1000 characters with an overlap of 200 characters
text_chunks = chunk_text(extracted_text, 1000, 200)

# Print the number of text chunks created
print("Number of text chunks:", len(text_chunks))

# Print the first text chunk
print("\nFirst text chunk:")
print(text_chunks[0])

Number of text chunks: 7

First text chunk:
Pahalgam attack: Prime Minister Narendra Modi arrived in New Delhi on Wednesday morning after cutting his Saudi trip short due to the terror attack in Jammu and Kashmir's Pahalgam. He had a brief meeting with NSA Ajit Doval, external affairs minister S Jaishankar, and foreign secretary Vikram Misri at the airport.

At least 26 tourists have been killed after unidentified gunmen opened fire on innocent civilians in Pahalgam's Baisaran in Jammu and Kashmir. Gunshots were heard in the area, following which security forces rushed there.

Initial reports suggested a possible terror attack at a site frequented by tourists, the police said. Security forces have been rushed to the area, and an operation is currently underway, reported PTI.

The incident occurred at around 3 pm when terrorists came down from the mountain in Baisaran valley and started firing at the tourists who frequent the place, which is often dubbed as 'mini Switzerland' because of

In [ ]:
response = create_embeddings(text_chunks)

In [ ]:
def cosine_similarity(vec1, vec2):
    """
    Calculates the cosine similarity between two vectors.

    Args:
    vec1 (np.ndarray): The first vector.
    vec2 (np.ndarray): The second vector.

    Returns:
    float: The cosine similarity between the two vectors.
    """
    # Compute the dot product of the two vectors and divide by the product of their norms
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
def semantic_search(query, text_chunks, embeddings, k=5):
    """
    Performs semantic search on the text chunks using the given query and embeddings.

    Args:
    query (str): The query for the semantic search.
    text_chunks (List[str]): A list of text chunks to search through.
    embeddings (List[dict]): A list of embeddings for the text chunks.
    k (int): The number of top relevant text chunks to return. Default is 5.

    Returns:
    List[str]: A list of the top k most relevant text chunks based on the query.
    """
    # Create an embedding for the query
    query_embedding = create_embeddings(query)['embeddings']
    similarity_scores = []  # Initialize a list to store similarity scores

    # Calculate similarity scores between the query embedding and each text chunk embedding
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding))
        similarity_scores.append((i, similarity_score))  # Append the index and similarity score

    # Sort the similarity scores in descending order
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    # Get the indices of the top k most similar text chunks
    top_indices = [index for index, _ in similarity_scores[:k]]
    # Return the top k most relevant text chunks
    return [text_chunks[index] for index in top_indices]

In [ ]:
query = "Tell me about Indus treaty"

# Perform semantic search to find the top 2 most relevant text chunks for the query
top_chunks = semantic_search(query, text_chunks, response['embeddings'], k=2)

# Print the query
print("Query:", query)

# Print the top 2 most relevant text chunks
for i, chunk in enumerate(top_chunks):
    print(f"Context {i + 1}:\n{chunk}\n=====================================")

Query: Tell me about Indus treaty
Context 1:
isit to the US and Peru and will return to India at the earliest, according to an official statement released on Wednesday. Sitharaman had arrived in the US on Sunday for a six-day visit and was supposed to travel to Peru afterwards for a five-day trip.

The Cabinet Committee on Security (CCS), chaired by PM Modi, has decided on five key measures in the aftermath of the attack. They are:

(i) The Indus Waters Treaty of 1960 will be held in abeyance with immediate effect until Pakistan credibly and irrevocably abjures its support for cross-border terrorism.

(ii) The Integrated Check Post Attari will be closed with immediate effect. Those who have crossed over with valid endorsements may return through that route before 01 May 2025.

(iii) Pakistani nationals will not be permitted to travel to India under the SAARC Visa Exemption Scheme (SVES) visas. Any SVES visas issued in the past to Pakistani nationals are deemed cancelled. Any Pakistani 

In [ ]:
import huggingface_hub
from google.colab import userdata
huggingface_hub.login(token=userdata.get('HF_TOKEN'))

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

# You can use Huggingface in case you don't want to use gemini - specify the model & give it a shot
def generate_response(system_prompt, user_message, model_name="meta-llama/Llama-3.2-3B-Instruct"):
    """
    Generates a response from a Hugging Face model based on the system prompt and user message.

    Args:
    system_prompt (str): The system prompt to guide the AI's behavior.
    user_message (str): The user's message or query.
    model_name (str): The model to be used for generating the response. Default is "meta-llama/Llama-3.2-3B-Instruct".

    Returns:
    str: The response from the AI model.
    """
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Combine the system prompt and user message
    input_text = f"{system_prompt}\n{user_message}"

    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

    # Generate the response
    outputs = model.generate(inputs['input_ids'], max_length=500, num_return_sequences=1, temperature=0)

    # Decode the response and return it
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response_text



In [ ]:
from google import genai
def generate_response(system_prompt, user_message):
  client = genai.Client(api_key=userdata.get("gemini_api"))

  response = client.models.generate_content(
      model="gemini-2.0-flash",
      contents=[system_prompt+user_message]
  )
  return (response.text)

In [ ]:
# Create the user prompt based on the top chunks
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# Generate AI response
ai_response = generate_response(system_prompt, user_prompt)
print(ai_response)

The Indus Waters Treaty of 1960 will be held in abeyance with immediate effect until Pakistan credibly and irrevocably abjures its support for cross-border terrorism.

